In [125]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import re
import nltk
import matplotlib.pyplot as plt

In [4]:
#Upload the dataset in df
df=pd.read_csv('tweets.csv',nrows=500)

In [5]:
df

,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source
0,Bohmle,"#GPT4 for FREE. \nNo its not a clickbait, @Qol...",Carkingga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AI enthusiast,2019-07-03 03:44:41+00:00,60.0,349.0,611.0,False,2023-05-17 18:11:12+00:00,"['GPT4', 'ChatGPT4']",Twitter Web App,NaN,NaN
2,Dan Bruno AI,ChatGPT Thinks These 5 Crypto Coins Will Explo...,"Manchester, NH","The latest in #ChatGPT, #BARD, #Bing, and othe...",2021-05-19 01:19:32+00:00,470.0,157.0,5185,False,2023-05-17 18:11:03+00:00,"['chatgpt', 'AI', 'openAI']",dlvr.it
3,Georgiana Comsa,New: @JWVance's post about 5 #startups (includ...,Palo Alto,"Founder of Silicon Valley PR, award-winning PR...",2008-12-24 09:32:23+00:00,3864.0,1883.0,2415,False,2023-05-17 18:10:25+00:00,"['startups', 'startup50']",Twitter Web App
4,Bitone Great,🚨Get Out!🚨\n💰#Binance Spot💰\n⬇ Recommendation:...,Hong Kong,#ChatGPT (AI) powered Free Trading Signal! \nL...,2022-11-21 04:42:18+00:00,1517.0,506.0,64,False,2023-05-17 18:09:39+00:00,"['Binance', 'Short', 'GHSTUSDT']",rsi1
...,...,...,...,...,...,...,...,...,...,...,...,...
495,O'Dwyer's PR News,#ChatGPT set off an endless stream of fear and...,"New York, NY","The #1 news outlet for public relations, publi...",2009-02-25 15:50:27+00:00,15013.0,2876.0,1,False,2023-05-17 16:00:37+00:00,['ChatGPT'],Twitter Web App
496,"Tracy Krauss, #author #artist #playwright",ChatGPT is a hot topic; there is no doubt. It ...,"British Columbia, Canada",I write modern Christian romance with a twist ...,2010-06-19 04:45:43+00:00,2718.0,1803.0,348,False,2023-05-17 16:00:37+00:00,NaN,Hootsuite Inc.
497,Tom Solid,Exciting news! #chatGPT has launched its Plugi...,Germany,🚀✨ We help you to build your productivity syst...,2010-12-04 11:00:53+00:00,3460.0,332.0,4464,False,2023-05-17 16:00:30+00:00,['chatGPT'],Hootsuite Inc.
498,Tweeting Monk,@Oncrawl Agreed! The #ChatGPT is definitely th...,GET 30% OFF FOR 3 MONTHS,Set your Twitter account on auto-pilot and wat...,2022-03-30 10:35:13+00:00,50.0,111.0,64,False,2023-05-17 16:00:28+00:00,['ChatGPT'],TweetingMonk


# Q.1: EDA

In [8]:
#Finding total null values in the df
df.isnull().sum().sum()

276

In [11]:
columnNames=df.columns
columnNames

Index(['user_name', 'text', 'user_location', 'user_description',
       'user_created', 'user_followers', 'user_friends', 'user_favourites',
       'user_verified', 'date', 'hashtags', 'source'],
      dtype='object')

In [13]:
for colName in columnNames:
    nullValues=df[colName].isnull().sum()
    print(colName+' : ',nullValues)

user_name :  1
text :  0
user_location :  98
user_description :  27
user_created :  1
user_followers :  1
user_friends :  1
user_favourites :  1
user_verified :  1
date :  1
hashtags :  142
source :  2


In [15]:
df['hashtags']

0                                   NaN
1                                   NaN
2           ['chatgpt', 'AI', 'openAI']
3             ['startups', 'startup50']
4      ['Binance', 'Short', 'GHSTUSDT']
                     ...               
495                         ['ChatGPT']
496                                 NaN
497                         ['chatGPT']
498                         ['ChatGPT']
499                                 NaN
Name: hashtags, Length: 500, dtype: object

In [17]:
df.corr(numeric_only=True)

,user_followers
user_followers,1.0


# Q.2: Pre-processing & Cleaning

### Removing duplicate rows if any.

In [42]:
df2 = df.drop_duplicates()

In [43]:
df2

,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source
0,Bohmle,"#GPT4 for FREE. \nNo its not a clickbait, @Qol...",Carkingga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AI enthusiast,2019-07-03 03:44:41+00:00,60.0,349.0,611.0,False,2023-05-17 18:11:12+00:00,"['GPT4', 'ChatGPT4']",Twitter Web App,NaN,NaN
2,Dan Bruno AI,ChatGPT Thinks These 5 Crypto Coins Will Explo...,"Manchester, NH","The latest in #ChatGPT, #BARD, #Bing, and othe...",2021-05-19 01:19:32+00:00,470.0,157.0,5185,False,2023-05-17 18:11:03+00:00,"['chatgpt', 'AI', 'openAI']",dlvr.it
3,Georgiana Comsa,New: @JWVance's post about 5 #startups (includ...,Palo Alto,"Founder of Silicon Valley PR, award-winning PR...",2008-12-24 09:32:23+00:00,3864.0,1883.0,2415,False,2023-05-17 18:10:25+00:00,"['startups', 'startup50']",Twitter Web App
4,Bitone Great,🚨Get Out!🚨\n💰#Binance Spot💰\n⬇ Recommendation:...,Hong Kong,#ChatGPT (AI) powered Free Trading Signal! \nL...,2022-11-21 04:42:18+00:00,1517.0,506.0,64,False,2023-05-17 18:09:39+00:00,"['Binance', 'Short', 'GHSTUSDT']",rsi1
...,...,...,...,...,...,...,...,...,...,...,...,...
495,O'Dwyer's PR News,#ChatGPT set off an endless stream of fear and...,"New York, NY","The #1 news outlet for public relations, publi...",2009-02-25 15:50:27+00:00,15013.0,2876.0,1,False,2023-05-17 16:00:37+00:00,['ChatGPT'],Twitter Web App
496,"Tracy Krauss, #author #artist #playwright",ChatGPT is a hot topic; there is no doubt. It ...,"British Columbia, Canada",I write modern Christian romance with a twist ...,2010-06-19 04:45:43+00:00,2718.0,1803.0,348,False,2023-05-17 16:00:37+00:00,NaN,Hootsuite Inc.
497,Tom Solid,Exciting news! #chatGPT has launched its Plugi...,Germany,🚀✨ We help you to build your productivity syst...,2010-12-04 11:00:53+00:00,3460.0,332.0,4464,False,2023-05-17 16:00:30+00:00,['chatGPT'],Hootsuite Inc.
498,Tweeting Monk,@Oncrawl Agreed! The #ChatGPT is definitely th...,GET 30% OFF FOR 3 MONTHS,Set your Twitter account on auto-pilot and wat...,2022-03-30 10:35:13+00:00,50.0,111.0,64,False,2023-05-17 16:00:28+00:00,['ChatGPT'],TweetingMonk


In [44]:
#No duplicate rows in the dataframe.

### Removing rows with missing values.

In [45]:
df2=df2.dropna(axis=0, how="any")
df2

,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source
2,Dan Bruno AI,ChatGPT Thinks These 5 Crypto Coins Will Explo...,"Manchester, NH","The latest in #ChatGPT, #BARD, #Bing, and othe...",2021-05-19 01:19:32+00:00,470.0,157.0,5185,False,2023-05-17 18:11:03+00:00,"['chatgpt', 'AI', 'openAI']",dlvr.it
3,Georgiana Comsa,New: @JWVance's post about 5 #startups (includ...,Palo Alto,"Founder of Silicon Valley PR, award-winning PR...",2008-12-24 09:32:23+00:00,3864.0,1883.0,2415,False,2023-05-17 18:10:25+00:00,"['startups', 'startup50']",Twitter Web App
4,Bitone Great,🚨Get Out!🚨\n💰#Binance Spot💰\n⬇ Recommendation:...,Hong Kong,#ChatGPT (AI) powered Free Trading Signal! \nL...,2022-11-21 04:42:18+00:00,1517.0,506.0,64,False,2023-05-17 18:09:39+00:00,"['Binance', 'Short', 'GHSTUSDT']",rsi1
6,ScrapeLogo: acquired by RiteKit in 2015,Video : Show a company's logo based on the web...,"Prague, Czech Republic",Test RiteKit #RESTAPI solutions in demo page. ...,2015-01-31 04:13:07+00:00,176.0,54.0,274,False,2023-05-17 18:09:04+00:00,['Short'],RiteKit App
7,vigilance.eth™️ 0x676D | 🇨🇦🇮🇪🇬🇧🇨🇵,I just used my #CerberusGPT HologeneticsLab an...,"London, ON",🇨🇦 #blockchain #ai #ChatGPT #promptcraft 🤖wiza...,2021-01-30 21:03:36+00:00,123.0,299.0,8938,False,2023-05-17 18:09:00+00:00,['CerberusGPT'],Twitter Web App
...,...,...,...,...,...,...,...,...,...,...,...,...
493,Kristina Lucrezia Cornèr,"am i that lucky, or #chatGPT constantly makes ...","Veneto, Italy","See beyond, question more, believe in impossib...",2018-02-12 12:17:29+00:00,3028.0,483.0,1103,False,2023-05-17 16:00:54+00:00,['chatGPT'],Twitter Web App
494,Janice Portal | AI Enthusiast | Virtual Assistant,"Hi everyone, just want to share again my AI-ge...","Boracay, Philippines","Part-time Freelancer, Crypto & AI Enthusiast, ...",2023-03-07 16:18:50+00:00,14.0,74.0,33,False,2023-05-17 16:00:40+00:00,"['ChatGPT', 'OPENAI']",Twitter Web App
495,O'Dwyer's PR News,#ChatGPT set off an endless stream of fear and...,"New York, NY","The #1 news outlet for public relations, publi...",2009-02-25 15:50:27+00:00,15013.0,2876.0,1,False,2023-05-17 16:00:37+00:00,['ChatGPT'],Twitter Web App
497,Tom Solid,Exciting news! #chatGPT has launched its Plugi...,Germany,🚀✨ We help you to build your productivity syst...,2010-12-04 11:00:53+00:00,3460.0,332.0,4464,False,2023-05-17 16:00:30+00:00,['chatGPT'],Hootsuite Inc.


In [46]:
# Checking if any null value is left in the dataframe.
df2.isnull().sum().sum()

0

### Normalizing the data

In [47]:
type(df2['user_followers'])

pandas.core.series.Series

In [52]:
df2['user_favourites']=df2['user_favourites'].astype(float)

/var/folders/h9/5rbmcw493f30vqzlns2tt0sc0000gn/T/ipykernel_960/3716584358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['user_favourites']=df2['user_favourites'].astype(float)


In [59]:
df2.dtypes

user_name            object
text                 object
user_location        object
user_description     object
user_created         object
user_followers      float64
user_friends        float64
user_favourites     float64
user_verified        object
date                 object
hashtags             object
source               object
dtype: object

In [61]:
df_min_max_scaled = df2.copy()
  
# apply normalization techniques by Column 1
column1 = 'user_followers'
column2 = 'user_friends'
column3 = 'user_favourites'
df_min_max_scaled[column1] = (df_min_max_scaled[column1] - df_min_max_scaled[column1].min()) / (df_min_max_scaled[column1].max() - df_min_max_scaled[column1].min())
df_min_max_scaled[column2] = (df_min_max_scaled[column2] - df_min_max_scaled[column2].min()) / (df_min_max_scaled[column2].max() - df_min_max_scaled[column2].min())
df_min_max_scaled[column3] = (df_min_max_scaled[column3] - df_min_max_scaled[column3].min()) / (df_min_max_scaled[column3].max() - df_min_max_scaled[column3].min())
  
# view normalized data
display(df_min_max_scaled)

,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source
2,Dan Bruno AI,ChatGPT Thinks These 5 Crypto Coins Will Explo...,"Manchester, NH","The latest in #ChatGPT, #BARD, #Bing, and othe...",2021-05-19 01:19:32+00:00,0.000221,0.001395,0.014656,False,2023-05-17 18:11:03+00:00,"['chatgpt', 'AI', 'openAI']",dlvr.it
3,Georgiana Comsa,New: @JWVance's post about 5 #startups (includ...,Palo Alto,"Founder of Silicon Valley PR, award-winning PR...",2008-12-24 09:32:23+00:00,0.001823,0.016732,0.006826,False,2023-05-17 18:10:25+00:00,"['startups', 'startup50']",Twitter Web App
4,Bitone Great,🚨Get Out!🚨\n💰#Binance Spot💰\n⬇ Recommendation:...,Hong Kong,#ChatGPT (AI) powered Free Trading Signal! \nL...,2022-11-21 04:42:18+00:00,0.000716,0.004496,0.000181,False,2023-05-17 18:09:39+00:00,"['Binance', 'Short', 'GHSTUSDT']",rsi1
6,ScrapeLogo: acquired by RiteKit in 2015,Video : Show a company's logo based on the web...,"Prague, Czech Republic",Test RiteKit #RESTAPI solutions in demo page. ...,2015-01-31 04:13:07+00:00,0.000083,0.000480,0.000774,False,2023-05-17 18:09:04+00:00,['Short'],RiteKit App
7,vigilance.eth™️ 0x676D | 🇨🇦🇮🇪🇬🇧🇨🇵,I just used my #CerberusGPT HologeneticsLab an...,"London, ON",🇨🇦 #blockchain #ai #ChatGPT #promptcraft 🤖wiza...,2021-01-30 21:03:36+00:00,0.000058,0.002657,0.025264,False,2023-05-17 18:09:00+00:00,['CerberusGPT'],Twitter Web App
...,...,...,...,...,...,...,...,...,...,...,...,...
493,Kristina Lucrezia Cornèr,"am i that lucky, or #chatGPT constantly makes ...","Veneto, Italy","See beyond, question more, believe in impossib...",2018-02-12 12:17:29+00:00,0.001429,0.004292,0.003118,False,2023-05-17 16:00:54+00:00,['chatGPT'],Twitter Web App
494,Janice Portal | AI Enthusiast | Virtual Assistant,"Hi everyone, just want to share again my AI-ge...","Boracay, Philippines","Part-time Freelancer, Crypto & AI Enthusiast, ...",2023-03-07 16:18:50+00:00,0.000006,0.000658,0.000093,False,2023-05-17 16:00:40+00:00,"['ChatGPT', 'OPENAI']",Twitter Web App
495,O'Dwyer's PR News,#ChatGPT set off an endless stream of fear and...,"New York, NY","The #1 news outlet for public relations, publi...",2009-02-25 15:50:27+00:00,0.007085,0.025556,0.000003,False,2023-05-17 16:00:37+00:00,['ChatGPT'],Twitter Web App
497,Tom Solid,Exciting news! #chatGPT has launched its Plugi...,Germany,🚀✨ We help you to build your productivity syst...,2010-12-04 11:00:53+00:00,0.001633,0.002950,0.012618,False,2023-05-17 16:00:30+00:00,['chatGPT'],Hootsuite Inc.


In [63]:
df3=df_min_max_scaled
df3[text]

### Removing stopwords from text

In [71]:
text=df3['text'].to_numpy()
text[1]

"New: @JWVance's post about 5 #startups (including @VcinityInc) that won the 1st #startup50 challenge (thx to #ChatGPT), based on answers about macro-economic pressures, VC funding, track record of management team &amp; growth potential of the market sector.  https://t.co/JXG37AMjh5"

In [76]:
def listToString(s):
    str1 = " "
    return (str1.join(s))
textt=listToString(text)
textt

'ChatGPT Thinks These 5 Crypto Coins Will Explode This Year - Yahoo Finance #chatgpt #AI #openAI https://t.co/tFOdU0qnDz New: @JWVance\'s post about 5 #startups (including @VcinityInc) that won the 1st #startup50 challenge (thx to #ChatGPT), based on answers about macro-economic pressures, VC funding, track record of management team &amp; growth potential of the market sector.  https://t.co/JXG37AMjh5 🚨Get Out!🚨\n💰#Binance Spot💰\n⬇ Recommendation: #Short 🔴\nTicker:  #GHSTUSDT\nTime Interval:  5min\nLast Price: 0.956\n🔴 RSI: 88.3\n\nPowered by #ChatGPT\n\n$GHST\n#GHST\n#AlphaQuarkToken\n#저스트\n\nWhat are you gonna do?\n👇LONG        👇WAIT         👇SHORT https://t.co/QIqgtT11Aa Video : Show a company\'s logo based on the website domain | #Short 💻 📹 - https://t.co/swo1GNqbEa 👈🏼Get the #CompanyLogo #API that does what #chatGPT cannot do https://t.co/vSLBWtAtfx I just used my #CerberusGPT HologeneticsLab and my simulated Hologenetic digital-organic supercomputer to create an internal revision

In [84]:
wordTokens = word_tokenize(textt)
len(wordTokens)

9992

In [83]:
stop_words = set(stopwords.words('english'))
for i in range(len(wordTokens)):
#    example_sent = df['review'][i]
#    word_tokens = word_tokenize(example_sent)
    filtered_sentence = [w for w in wordTokens if not w.lower() in stop_words]
#with no lower case conversion
    filtered_sentence = []
  
    for w in wordTokens:
        if w not in stop_words:
            filtered_sentence.append(w)
print(filtered_sentence)

['ChatGPT', 'Thinks', 'These', '5', 'Crypto', 'Coins', 'Will', 'Explode', 'This', 'Year', '-', 'Yahoo', 'Finance', '#', 'chatgpt', '#', 'AI', '#', 'openAI', 'https', ':', '//t.co/tFOdU0qnDz', 'New', ':', '@', 'JWVance', "'s", 'post', '5', '#', 'startups', '(', 'including', '@', 'VcinityInc', ')', '1st', '#', 'startup50', 'challenge', '(', 'thx', '#', 'ChatGPT', ')', ',', 'based', 'answers', 'macro-economic', 'pressures', ',', 'VC', 'funding', ',', 'track', 'record', 'management', 'team', '&', 'amp', ';', 'growth', 'potential', 'market', 'sector', '.', 'https', ':', '//t.co/JXG37AMjh5', '🚨Get', 'Out', '!', '🚨', '💰', '#', 'Binance', 'Spot💰', '⬇', 'Recommendation', ':', '#', 'Short', '🔴', 'Ticker', ':', '#', 'GHSTUSDT', 'Time', 'Interval', ':', '5min', 'Last', 'Price', ':', '0.956', '🔴', 'RSI', ':', '88.3', 'Powered', '#', 'ChatGPT', '$', 'GHST', '#', 'GHST', '#', 'AlphaQuarkToken', '#', '저스트', 'What', 'gon', 'na', '?', '👇LONG', '👇WAIT', '👇SHORT', 'https', ':', '//t.co/QIqgtT11Aa', 'Video

In [85]:
len(filtered_sentence)

8169

### Stemming

In [ ]:
ps = PorterStemmer()
for i in range(len(filtered_sentence)):
    #review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    #review = review.lower()
    #review = review.split()
    review = [ps.stem(word) for word in review]
    #review = ' '.join(review)
    #corpus.append(review)

In [94]:
ps = PorterStemmer()
filtered_words=[]
for word in filtered_sentence:
    filtered_words.append(ps.stem(word))
filtered_words

['chatgpt',
 'think',
 'these',
 '5',
 'crypto',
 'coin',
 'will',
 'explod',
 'thi',
 'year',
 '-',
 'yahoo',
 'financ',
 '#',
 'chatgpt',
 '#',
 'ai',
 '#',
 'openai',
 'http',
 ':',
 '//t.co/tfodu0qndz',
 'new',
 ':',
 '@',
 'jwvanc',
 "'s",
 'post',
 '5',
 '#',
 'startup',
 '(',
 'includ',
 '@',
 'vcinityinc',
 ')',
 '1st',
 '#',
 'startup50',
 'challeng',
 '(',
 'thx',
 '#',
 'chatgpt',
 ')',
 ',',
 'base',
 'answer',
 'macro-econom',
 'pressur',
 ',',
 'vc',
 'fund',
 ',',
 'track',
 'record',
 'manag',
 'team',
 '&',
 'amp',
 ';',
 'growth',
 'potenti',
 'market',
 'sector',
 '.',
 'http',
 ':',
 '//t.co/jxg37amjh5',
 '🚨get',
 'out',
 '!',
 '🚨',
 '💰',
 '#',
 'binanc',
 'spot💰',
 '⬇',
 'recommend',
 ':',
 '#',
 'short',
 '🔴',
 'ticker',
 ':',
 '#',
 'ghstusdt',
 'time',
 'interv',
 ':',
 '5min',
 'last',
 'price',
 ':',
 '0.956',
 '🔴',
 'rsi',
 ':',
 '88.3',
 'power',
 '#',
 'chatgpt',
 '$',
 'ghst',
 '#',
 'ghst',
 '#',
 'alphaquarktoken',
 '#',
 '저스트',
 'what',
 'gon',
 'na',
 

### Lemmetization

In [96]:
lem = WordNetLemmatizer()
filtered_words=[]
filtered_document=''
for word in filtered_sentence:
    filtered_words.append(lem.lemmatize(word))
    filtered_document = ' '.join(filtered_words)
filtered_words

['ChatGPT',
 'Thinks',
 'These',
 '5',
 'Crypto',
 'Coins',
 'Will',
 'Explode',
 'This',
 'Year',
 '-',
 'Yahoo',
 'Finance',
 '#',
 'chatgpt',
 '#',
 'AI',
 '#',
 'openAI',
 'http',
 ':',
 '//t.co/tFOdU0qnDz',
 'New',
 ':',
 '@',
 'JWVance',
 "'s",
 'post',
 '5',
 '#',
 'startup',
 '(',
 'including',
 '@',
 'VcinityInc',
 ')',
 '1st',
 '#',
 'startup50',
 'challenge',
 '(',
 'thx',
 '#',
 'ChatGPT',
 ')',
 ',',
 'based',
 'answer',
 'macro-economic',
 'pressure',
 ',',
 'VC',
 'funding',
 ',',
 'track',
 'record',
 'management',
 'team',
 '&',
 'amp',
 ';',
 'growth',
 'potential',
 'market',
 'sector',
 '.',
 'http',
 ':',
 '//t.co/JXG37AMjh5',
 '🚨Get',
 'Out',
 '!',
 '🚨',
 '💰',
 '#',
 'Binance',
 'Spot💰',
 '⬇',
 'Recommendation',
 ':',
 '#',
 'Short',
 '🔴',
 'Ticker',
 ':',
 '#',
 'GHSTUSDT',
 'Time',
 'Interval',
 ':',
 '5min',
 'Last',
 'Price',
 ':',
 '0.956',
 '🔴',
 'RSI',
 ':',
 '88.3',
 'Powered',
 '#',
 'ChatGPT',
 '$',
 'GHST',
 '#',
 'GHST',
 '#',
 'AlphaQuarkToken',
 '#',

### Removing Punctuation

In [97]:
filtered_document

"ChatGPT Thinks These 5 Crypto Coins Will Explode This Year - Yahoo Finance # chatgpt # AI # openAI http : //t.co/tFOdU0qnDz New : @ JWVance 's post 5 # startup ( including @ VcinityInc ) 1st # startup50 challenge ( thx # ChatGPT ) , based answer macro-economic pressure , VC funding , track record management team & amp ; growth potential market sector . http : //t.co/JXG37AMjh5 🚨Get Out ! 🚨 💰 # Binance Spot💰 ⬇ Recommendation : # Short 🔴 Ticker : # GHSTUSDT Time Interval : 5min Last Price : 0.956 🔴 RSI : 88.3 Powered # ChatGPT $ GHST # GHST # AlphaQuarkToken # 저스트 What gon na ? 👇LONG 👇WAIT 👇SHORT http : //t.co/QIqgtT11Aa Video : Show company 's logo based website domain | # Short 💻 📹 - http : //t.co/swo1GNqbEa 👈🏼Get # CompanyLogo # API # chatGPT http : //t.co/vSLBWtAtfx I used # CerberusGPT HologeneticsLab simulated Hologenetic digital-organic supercomputer create internal revision system # ChatGPT based organic process . Even system subject security breach ( likely ) prompt injection o

In [101]:
filtered_punc=[]
for word in filtered_words:
    if word in string.punctuation:
        word=''
    filtered_punc.append(word)
    filtered_document = ' '.join(filtered_punc)

In [102]:
filtered_document

"ChatGPT Thinks These 5 Crypto Coins Will Explode This Year  Yahoo Finance  chatgpt  AI  openAI http  //t.co/tFOdU0qnDz New   JWVance 's post 5  startup  including  VcinityInc  1st  startup50 challenge  thx  ChatGPT   based answer macro-economic pressure  VC funding  track record management team  amp  growth potential market sector  http  //t.co/JXG37AMjh5 🚨Get Out  🚨 💰  Binance Spot💰 ⬇ Recommendation   Short 🔴 Ticker   GHSTUSDT Time Interval  5min Last Price  0.956 🔴 RSI  88.3 Powered  ChatGPT  GHST  GHST  AlphaQuarkToken  저스트 What gon na  👇LONG 👇WAIT 👇SHORT http  //t.co/QIqgtT11Aa Video  Show company 's logo based website domain   Short 💻 📹  http  //t.co/swo1GNqbEa 👈🏼Get  CompanyLogo  API  chatGPT http  //t.co/vSLBWtAtfx I used  CerberusGPT HologeneticsLab simulated Hologenetic digital-organic supercomputer create internal revision system  ChatGPT based organic process  Even system subject security breach  likely  prompt injection or… http  //t.co/Aa9oxdV7qt 🚨Time Sell  🚨 💰  Binance 

### Removing links

In [107]:
#links remove nai hoi
import re
s1 = re.sub('http://\S+|https://\S+', '', filtered_document)
s1



"ChatGPT Thinks These 5 Crypto Coins Will Explode This Year  Yahoo Finance  chatgpt  AI  openAI http  //t.co/tFOdU0qnDz New   JWVance 's post 5  startup  including  VcinityInc  1st  startup50 challenge  thx  ChatGPT   based answer macro-economic pressure  VC funding  track record management team  amp  growth potential market sector  http  //t.co/JXG37AMjh5 🚨Get Out  🚨 💰  Binance Spot💰 ⬇ Recommendation   Short 🔴 Ticker   GHSTUSDT Time Interval  5min Last Price  0.956 🔴 RSI  88.3 Powered  ChatGPT  GHST  GHST  AlphaQuarkToken  저스트 What gon na  👇LONG 👇WAIT 👇SHORT http  //t.co/QIqgtT11Aa Video  Show company 's logo based website domain   Short 💻 📹  http  //t.co/swo1GNqbEa 👈🏼Get  CompanyLogo  API  chatGPT http  //t.co/vSLBWtAtfx I used  CerberusGPT HologeneticsLab simulated Hologenetic digital-organic supercomputer create internal revision system  ChatGPT based organic process  Even system subject security breach  likely  prompt injection or… http  //t.co/Aa9oxdV7qt 🚨Time Sell  🚨 💰  Binance 

In [110]:
lst = re.findall('[http]\s+[.+]\s+', filtered_document)
for i in lst:
    s1 = filtered_document.replace(i, '')
s1

"ChatGPT Thinks These 5 Crypto Coins Will Explode This Year  Yahoo Finance  chatgpt  AI  openAI http  //t.co/tFOdU0qnDz New   JWVance 's post 5  startup  including  VcinityInc  1st  startup50 challenge  thx  ChatGPT   based answer macro-economic pressure  VC funding  track record management team  amp  growth potential market sector  http  //t.co/JXG37AMjh5 🚨Get Out  🚨 💰  Binance Spot💰 ⬇ Recommendation   Short 🔴 Ticker   GHSTUSDT Time Interval  5min Last Price  0.956 🔴 RSI  88.3 Powered  ChatGPT  GHST  GHST  AlphaQuarkToken  저스트 What gon na  👇LONG 👇WAIT 👇SHORT http  //t.co/QIqgtT11Aa Video  Show company 's logo based website domain   Short 💻 📹  http  //t.co/swo1GNqbEa 👈🏼Get  CompanyLogo  API  chatGPT http  //t.co/vSLBWtAtfx I used  CerberusGPT HologeneticsLab simulated Hologenetic digital-organic supercomputer create internal revision system  ChatGPT based organic process  Even system subject security breach  likely  prompt injection or… http  //t.co/Aa9oxdV7qt 🚨Time Sell  🚨 💰  Binance 

### Removing special characters

In [108]:
def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    return text

In [116]:
remove_special_characters(s1)

'ChatGPT Thinks These 5 Crypto Coins Will Explode This Year  Yahoo Finance  chatgpt  AI  openAI http  tcotFOdU0qnDz New   JWVance s post 5  startup  including  VcinityInc  1st  startup50 challenge  thx  ChatGPT   based answer macroeconomic pressure  VC funding  track record management team  amp  growth potential market sector  http  tcoJXG37AMjh5 Get Out     Binance Spot  Recommendation   Short  Ticker   GHSTUSDT Time Interval  5min Last Price  0956  RSI  883 Powered  ChatGPT  GHST  GHST  AlphaQuarkToken   What gon na  LONG WAIT SHORT http  tcoQIqgtT11Aa Video  Show company s logo based website domain   Short    http  tcoswo1GNqbEa Get  CompanyLogo  API  chatGPT http  tcovSLBWtAtfx I used  CerberusGPT HologeneticsLab simulated Hologenetic digitalorganic supercomputer create internal revision system  ChatGPT based organic process  Even system subject security breach  likely  prompt injection or http  tcoAa9oxdV7qt Time Sell     Binance Spot  Recommendation   Short  Ticker   ALPACABUSD T

# N-Grams Analysis

In [122]:
tokens = nltk.word_tokenize(s1)
bigrams = nltk.bigrams(tokens)
trigrams = nltk.trigrams(tokens)

In [123]:
frequency = nltk.FreqDist(bigrams)
for key,value in frequency.items():
    print(key,value)

frequency2 = nltk.FreqDist(trigrams)
for key,value in frequency2.items():
    print(key,value)

('ChatGPT', 'Thinks') 1
('Thinks', 'These') 1
('These', '5') 1
('5', 'Crypto') 1
('Crypto', 'Coins') 1
('Coins', 'Will') 1
('Will', 'Explode') 1
('Explode', 'This') 1
('This', 'Year') 1
('Year', 'Yahoo') 1
('Yahoo', 'Finance') 1
('Finance', 'chatgpt') 1
('chatgpt', 'AI') 14
('AI', 'openAI') 13
('openAI', 'http') 13
('http', '//t.co/tFOdU0qnDz') 1
('//t.co/tFOdU0qnDz', 'New') 1
('New', 'JWVance') 1
('JWVance', "'s") 1
("'s", 'post') 1
('post', '5') 1
('5', 'startup') 1
('startup', 'including') 1
('including', 'VcinityInc') 1
('VcinityInc', '1st') 1
('1st', 'startup50') 1
('startup50', 'challenge') 1
('challenge', 'thx') 1
('thx', 'ChatGPT') 1
('ChatGPT', 'based') 2
('based', 'answer') 1
('answer', 'macro-economic') 1
('macro-economic', 'pressure') 1
('pressure', 'VC') 1
('VC', 'funding') 1
('funding', 'track') 1
('track', 'record') 1
('record', 'management') 1
('management', 'team') 1
('team', 'amp') 1
('amp', 'growth') 1
('growth', 'potential') 1
('potential', 'market') 1
('market', 's

In [119]:
frequency
from nltk.util import ngrams
from collections import Counter

In [130]:
bigram_counts = Counter(frequency)
top10_bigrams = bigram_counts.most_common(10)

top10_bigrams

[(('ChatGPT', 'http'), 21),
 (('Last', 'Price'), 20),
 (('Recommendation', 'Short'), 18),
 (('gon', 'na'), 17),
 (('🚨', '💰'), 15),
 (('💰', 'Binance'), 15),
 (('Binance', 'Spot💰'), 15),
 (('Spot💰', '⬇'), 15),
 (('⬇', 'Recommendation'), 15),
 (('Short', '🔴'), 15)]

In [124]:
trigram_counts = Counter(frequency2)
top10_trigrams = trigram_counts.most_common(10)

top10_trigrams

[(('🚨', '💰', 'Binance'), 15),
 (('💰', 'Binance', 'Spot💰'), 15),
 (('Binance', 'Spot💰', '⬇'), 15),
 (('Spot💰', '⬇', 'Recommendation'), 15),
 (('⬇', 'Recommendation', 'Short'), 15),
 (('Recommendation', 'Short', '🔴'), 15),
 (('Short', '🔴', 'Ticker'), 15),
 (('What', 'gon', 'na'), 15),
 (('gon', 'na', '👇LONG'), 15),
 (('na', '👇LONG', '👇WAIT'), 15)]

In [128]:
x = [str(bigrams) for bigrams in top10_bigrams.keys()]
y = list(top10_bigrams.values())
plt.bar(x, y)
plt.show()

AttributeError: 'list' object has no attribute 'keys'